# Creating a feature matrix from a DB Query

In [9]:
import sys
sys.path.append('/Users/celia/GitHub/mouse_bandit/data_preprocessing_code')
sys.path.append('/Users/celia/GitHub/mouse_bandit')
import support_functions as sf
import numpy as np
import pandas as pd
import scipy as sp
import bandit_preprocessing as bp
from sklearn import preprocessing
import sys
import os
import ipywidgets as widgets
from IPython.display import display
%matplotlib inline 

## Retrieve names of sessions you want from the DB

In [44]:
#load in data base
db = pd.read_csv('/Users/celia/GitHub/mouse_bandit/celia_session_record.csv',index_col=0)

In [45]:
db.shape

(262, 18)

In [41]:
db.head(35)

,Owner,Session ID,Mouse ID,Date,Phase,Markov,Left Reward Prob,Right Reward Prob,Block Range Min,Block Range Max,No. Trials,No. Blocks,No. Rewards,p(high Port),Decision Window Duration,Min Inter-trial-interval,Left Solenoid Duration,Right Solenoid Duration
0,celia,08032017_Baby,Baby,2017-08-03,0.0,0,1.0,1.0,1000.0,1000.0,3.0,0.0,0.0,0.00,10.0,1.0,50.0,50.0
1,celia,08042017_Baby,Baby,2017-08-04,1.0,0,0.9,0.9,1000.0,1000.0,19.0,0.0,15.0,0.00,15.0,1.0,60.0,60.0
2,celia,08072017_Baby,Baby,2017-08-07,1.0,0,0.9,0.9,1000.0,1000.0,60.0,0.0,51.0,0.00,15.0,1.0,50.0,50.0
3,celia,08082017_Baby,Baby,2017-08-08,1.0,0,0.9,0.9,1000.0,1000.0,76.0,0.0,71.0,0.00,10.0,1.0,50.0,50.0
4,celia,08092017_Baby,Baby,2017-08-09,1.0,0,0.9,0.9,1000.0,1000.0,108.0,0.0,96.0,0.00,5.0,1.0,45.0,45.0
5,celia,08102017_Baby,Baby,2017-08-10,2.0,0,0.9,0.1,50.0,50.0,122.0,0.0,41.0,0.35,5.0,1.0,50.0,50.0
6,celia,08112017_Baby,Baby,2017-08-11,2.0,0,0.1,0.9,50.0,50.0,159.0,1.0,84.0,0.52,5.0,1.0,50.0,50.0
7,celia,08142017_Baby,Baby,2017-08-14,2.0,0,0.1,0.9,50.0,50.0,230.0,2.0,128.0,0.58,5.0,1.0,45.0,45.0
8,celia,08152017_Baby,Baby,2017-08-15,2.0,0,0.1,0.9,50.0,50.0,421.0,5.0,252.0,0.62,4.0,1.0,40.0,40.0
9,celia,08162017_Baby,Baby,2017-08-16,2.0,0,0.1,0.9,50.0,50.0,208.0,2.0,145.0,0.75,3.0,1.0,45.0,45.0


### Query all (80-20) sessions where performance > 0.7 and number of blocks exceeded 0 (no error)

In [54]:
r = db[(db['Markov']==1)]
r = db[((db['Left Reward Prob'] == 0.9) |  (db['Left Reward Prob'] == 0.1))].copy()
r = r[r['p(high Port)'] > 0.7].copy()
r = r[r['No. Blocks']>0.0].copy()
#r = r[r['Mouse ID'] == 'Mustard']
session_names = r['Session ID'].values

In [55]:
r.shape

(76, 18)

In [56]:
#r.to_csv(os.path.join(save_dir,'trials_markov_9010.csv'),index=True)

## load in csv files (from running exportTrials.m)

In [24]:
'''
load in trial data
'''
columns = ['Elapsed Time (s)','Since last trial (s)','Trial Duration (s)','Port Poked',
           'Right Reward Prob','Left Reward Prob','Reward Given',
          'center_frame','decision_frame', 'Block ID']

root_dir = '/Users/celia/GitHub/mouse_bandit/data/trial_data'
save_dir = '/Users/celia/GitHub/mouse_bandit/data/processed_data'

trial_df = []

for session in session_names:
    full_name = session + '_trials.csv'
    
    path_name = os.path.join(root_dir,full_name)
    
    trial_df.append(pd.read_csv(path_name,names=columns))

mouse_ids = r['Mouse ID'].values

In [25]:
len(trial_df)

85

## convert into 1 feature matrix

In [26]:
progress = widgets.IntProgress(min=0, max=len(trial_df), step=1)
display(progress)
for i,df in enumerate(trial_df):
    curr_feature_matrix = bp.create_feature_matrix(df,10,mouse_ids[i],session_names[i],feature_names='Default')
    
    if i == 0:
        master_matrix = curr_feature_matrix.copy()
    else:
        master_matrix = master_matrix.append(curr_feature_matrix)
    progress.value+=1

In [ ]:
np.where(block_starts[:n_indi]==True)[0][0]

In [ ]:
block_starts

In [58]:
master_matrix.tail(5)

,Mouse ID,Session ID,Trial,Block Trial,Block Reward,Port Streak,Reward Streak,10_Port,10_Reward,10_ITI,...,1_Port,1_Reward,1_ITI,1_trialDuration,0_ITI,0_trialDuration,Decision,Switch,Higher p port,Reward
796,Wine,10162017_Wine,807.0,11.0,5.0,1.0,1.0,1.0,1.0,1.387,...,1.0,1.0,1.295,0.315,1.310,0.294,1.0,0.0,0.0,1.0
797,Wine,10162017_Wine,808.0,12.0,6.0,2.0,2.0,1.0,0.0,1.377,...,1.0,1.0,1.310,0.294,1.297,0.316,1.0,0.0,0.0,0.0
798,Wine,10162017_Wine,809.0,13.0,6.0,3.0,-1.0,1.0,0.0,1.372,...,1.0,0.0,1.297,0.316,1.323,0.506,1.0,0.0,0.0,0.0
799,Wine,10162017_Wine,810.0,14.0,6.0,4.0,-2.0,1.0,0.0,1.208,...,1.0,0.0,1.323,0.506,1.838,0.570,0.0,1.0,1.0,0.0
800,Wine,10162017_Wine,811.0,15.0,6.0,1.0,-3.0,1.0,0.0,12.070,...,0.0,0.0,1.838,0.570,1.327,0.462,1.0,1.0,0.0,0.0


In [59]:
master_matrix.shape

(63853, 53)

In [103]:
master_matrix['Higher p port'].mean()

0.8428510090167454

## Save combined feature matrix  

In [60]:
master_matrix.to_csv(os.path.join(save_dir,'markov_8020.csv'),index=True)